In [2]:
import importData as import_data
import torch
import torch.nn as nn
import torch.optim as optim
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
import numpy as np
import random
torch.backends.cuda.matmul.allow_tf32 = True

uuids = ["99AAC7DE-4106-43AB-AA80-F5816AB4DB85", "69508C29-2B15-4230-BE32-328E553EC63D", "2D2247C8-6B76-45BE-AC6B-F177A87ED4EF"]
id = import_data.importData()
d_final, t_final = None, None
for uuid in uuids:
    d, t = id.import_data(uuid, 0.02, 100,2)
    if d_final is None:
        d_final, t_final = d, t
    else:
        # print("o")
        if len(t_final[0]) == len(t[0]):
            d_final = np.concatenate((d_final, d), axis=0)
            t_final = np.concatenate((t_final, t), axis=0)
        else:
            if len(t_final[0]) < len(t[0]):
                difference = -len(t_final[0]) + len(t[0])
                lengthened_t_final = []
                for i, t_fin in enumerate(t_final):
                    # print(np.shape(t_fin))
                    zeros = np.zeros((difference,1))
                    # print(np.shape(zeros))
                    # print(t_fin)
                    # print(zeros)
                    lengthened_t_final.append(np.concatenate((t_fin, np.zeros((difference,1))), axis=0))
                d_final = np.concatenate((d_final, d), axis=0)
                t_final = np.concatenate((lengthened_t_final, t), axis=0)
            elif len(t_final[0]) > len(t[0]):
                difference = len(t_final[0]) - len(t[0])
                lengthened_t= []
                for i, t_fin in enumerate(t):
                    # print(np.shape(t_fin))
                    zeros = np.zeros((difference,1))
                    # print(np.shape(zeros))
                    # print(t_fin)
                    # print(zeros)
                    lengthened_t.append(np.concatenate((t_fin, np.zeros((difference,1))), axis=0))
                d_final = np.concatenate((d_final, d), axis=0)
                t_final = np.concatenate((t_final, lengthened_t), axis=0)
                



/Users/thomas/Development/AATraining/importData.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trimmed_raw_df["time"] = trimmed_raw_df["time"].apply(lambda x: x - timestamps_df["startTime"][0])
/Users/thomas/Development/AATraining/importData.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_time_dataset["time"] = start_time_dataset["time"] - start_time
/Users/thomas/Development/AATraining/importData.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

Could not find start time, skipping this dataset
Could not find start time, skipping this dataset
Could not find start time, skipping this dataset


/Users/thomas/Development/AATraining/importData.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trimmed_raw_df["time"] = trimmed_raw_df["time"].apply(lambda x: x - timestamps_df["startTime"][0])
/Users/thomas/Development/AATraining/importData.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_time_dataset["time"] = start_time_dataset["time"] - start_time


In [3]:
print(np.shape(t_final))
print((d_final[0][6])[:5])

(16431, 1, 1)
[[0.02537209]
 [0.02745974]
 [0.01572108]
 [0.00457269]
 [0.00149256]]


In [4]:
print(torch.__version__)

2.6.0


In [6]:
# Check if GPU is available
if   torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

# Splitting data into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(d_final, t_final, test_size=0.2, random_state=42)

# Verify that the split is done correctly
print(f"X_train shape: {np.shape(X_train)}")
print(f"Y_train shape: {np.shape(Y_train)}")
print(f"X_val shape: {np.shape(X_val)}")
print(f"Y_val shape: {np.shape(Y_val)}")

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
Y_train = torch.tensor(Y_train, dtype=torch.float32).to(device)
X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
Y_val = torch.tensor(Y_val, dtype=torch.float32).to(device)

# Create datasets and dataloaders
train_dataset = TensorDataset(X_train, Y_train)
val_dataset = TensorDataset(X_val, Y_val)

train_loader = DataLoader(train_dataset, batch_size=3000, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=3000)

# Define the model
class CNNLSTMModel(nn.Module):
    def __init__(self, max_seq_length):
        super(CNNLSTMModel, self).__init__()
        self.conv = nn.Conv2d(9, 128, kernel_size=(3, 1))
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=(2, 1))  # Only pool along the height dimension
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(12672, 128)  # Adjusted dimensions after conv and pool
        self.repeat = nn.Linear(128, 128 * max_seq_length)
        self.lstm = nn.LSTM(128, 64, batch_first=True)
        self.fc_out = nn.Linear(64, 1)

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.relu(self.fc(x))
        x = self.repeat(x)
        x = x.view(-1, max_seq_length, 128)  # Reshape to (batch_size, seq_length, input_size)
        x, _ = self.lstm(x)
        x = self.fc_out(x)
        return x

max_seq_length = Y_train.shape[1]
model = CNNLSTMModel(max_seq_length).to(device)

# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
print("Training model")
epochs = 100
for epoch in range(epochs):
    model.train()
    for X_batch, Y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, Y_batch)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, Y_batch in val_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, Y_batch)
            val_loss += loss.item()

    if (epoch+1) % 10  == 0:
        print(f"Epoch {epoch+1}/{epochs}, Training Loss: {loss.item()}, Validation Loss: {val_loss / len(val_loader)}")

print("Training complete")

Using device: mps
X_train shape: (13144, 9, 200, 1)
Y_train shape: (13144, 1, 1)
X_val shape: (3287, 9, 200, 1)
Y_val shape: (3287, 1, 1)
Training model
Epoch 10/100, Training Loss: 0.08313465863466263, Validation Loss: 0.08658119291067123
Epoch 20/100, Training Loss: 0.08534877747297287, Validation Loss: 0.08323398232460022
Epoch 30/100, Training Loss: 0.09372290968894958, Validation Loss: 0.0885239951312542
Epoch 40/100, Training Loss: 0.07976388186216354, Validation Loss: 0.0745658241212368
Epoch 50/100, Training Loss: 0.07867102324962616, Validation Loss: 0.07299650087952614
Epoch 60/100, Training Loss: 0.07204126566648483, Validation Loss: 0.06742030009627342
Epoch 70/100, Training Loss: 0.06890368461608887, Validation Loss: 0.06495996564626694
Epoch 80/100, Training Loss: 0.057045940309762955, Validation Loss: 0.0573909766972065
Epoch 90/100, Training Loss: 0.05835511162877083, Validation Loss: 0.05851021222770214
Epoch 100/100, Training Loss: 0.05261973664164543, Validation Loss

In [7]:
# Save the trained model
torch.save(model.state_dict(), "torch_model_2.pth")

# Convert the model to TorchScript
model.eval() 
# An example input tensor with the same dimensions as the input data
example_input = torch.rand(1, 9, 200, 1).to(device)  
traced_script_module = torch.jit.trace(model, example_input)

# Save the TorchScript model
traced_script_module.save("torch_trace_model_2.pt")

In [8]:
testuuid = "98006881-B2F0-4B6B-94CF-A21ABE69F21B"


d_test, t_test = id.import_data(testuuid, 0.02, 100, 2)
indices = list(range(len(d_test)))
sampled_indices = random.sample(indices, 1000)
sampled_indices = sorted(sampled_indices)
d_test, t_test = d_test[sampled_indices], t_test[sampled_indices]

new_data = torch.tensor(d_test, dtype=torch.float32).to(device)

# Load the saved model
model = CNNLSTMModel(max_seq_length).to(device)
model.load_state_dict(torch.load("torch_model_2.pth"))
model.eval()

# Run the model on the new dataset
with torch.no_grad():
    outputs = model(new_data)

# Convert the outputs to a NumPy array
outputs = outputs.cpu().numpy()

# Print the first few predictions
print("Model predictions on new dataset:")
print(outputs[:5])
print("test hypersuccess")

/Users/thomas/Development/AATraining/importData.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trimmed_raw_df["time"] = trimmed_raw_df["time"].apply(lambda x: x - timestamps_df["startTime"][0])
/Users/thomas/Development/AATraining/importData.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_time_dataset["time"] = start_time_dataset["time"] - start_time


Model predictions on new dataset:
[[[0.00675528]]

 [[0.00703042]]

 [[0.00774223]]

 [[0.01036542]]

 [[0.01069561]]]
test hypersuccess


In [9]:
for i, output in enumerate(outputs):
    print(output, t_test[i])

[[0.00675528]] [[0.]]
[[0.00703042]] [[0.]]
[[0.00774223]] [[0.]]
[[0.01036542]] [[0.]]
[[0.01069561]] [[0.]]
[[0.01018587]] [[0.]]
[[0.00804317]] [[0.]]
[[0.00707874]] [[0.]]
[[0.00568781]] [[0.]]
[[0.00563673]] [[0.]]
[[0.00566428]] [[0.]]
[[0.00922664]] [[0.]]
[[0.00997469]] [[0.]]
[[0.00546965]] [[0.]]
[[0.00655639]] [[0.]]
[[0.00617026]] [[0.]]
[[0.00527724]] [[0.]]
[[0.00554919]] [[0.]]
[[0.00548293]] [[0.]]
[[0.00535844]] [[0.]]
[[0.00434169]] [[0.]]
[[0.00403629]] [[0.]]
[[0.0030123]] [[0.]]
[[0.00582982]] [[0.]]
[[0.00541848]] [[0.]]
[[0.00480393]] [[0.]]
[[0.00564097]] [[0.]]
[[0.00768361]] [[0.]]
[[0.00738056]] [[0.]]
[[0.00570971]] [[0.]]
[[0.00142682]] [[0.]]
[[-0.00431833]] [[0.]]
[[-0.00385856]] [[0.]]
[[-0.00233629]] [[0.]]
[[0.00174347]] [[0.]]
[[0.0010265]] [[0.]]
[[-0.00077491]] [[0.]]
[[-0.00266827]] [[0.]]
[[-0.00979718]] [[0.]]
[[0.00872009]] [[0.]]
[[-0.0031431]] [[0.]]
[[-0.00253222]] [[0.]]
[[-0.00307516]] [[0.]]
[[-0.00163758]] [[0.]]
[[-0.00154263]] [[0.]]
[[